0. Let's mount our drive

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

1. Download and install OpenNMT-py 2.0



In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install OpenNMT-py

Let's install some libraries:

In [ ]:
!pip install sacremoses

In [5]:
MYCLASSFOLDER="SE_personal"
import os
os.environ['PATH'] += ":/content/drive/MyDrive/" + MYCLASSFOLDER + "/scripts"

2. Let's get some data. We can get any parallel data from, e.g. http://opus.nlpl.eu or you can download the ones I will use for this tutorial.

*   English <-> Bulgarian: https://ilk.uvt.nl/~shterion/data/EN-BG.zip
*   English <-> Dutch: https://ilk.uvt.nl/~shterion/data/EN-NL.zip


We should put the data in our drive, otherwise when the session expires we wil lneed to do the same thing again. That is also valid for all models and intermediate processed files.

In [ ]:
!MYCLASSFOLDER="SE_personal"
!mkdir /content/drive/MyDrive/$MYCLASSFOLDER/
!ls /content/drive/MyDrive/$MYCLASSFOLDER

In [7]:
cd /content/drive/MyDrive/$MYCLASSFOLDER

/content/drive/MyDrive/SE_personal


In [ ]:
!mkdir scripts 
!wget https://ilk.uvt.nl/~shterion/scripts/train_test_dev.py -O scripts/train_test_dev.py
!wget https://ilk.uvt.nl/~shterion/scripts/bpe.sh -O scripts/bpe.sh

Now let's download scripts for tokenisation and detokanisation:

In [ ]:
!wget https://ilk.uvt.nl/~shterion/scripts/tokenizer.perl -O scripts/tokenizer.perl
!wget https://ilk.uvt.nl/~shterion/scripts/detokenize.perl -O scripts/detokenize.perl
!wget https://ilk.uvt.nl/~shterion/scripts/nonbreaking_prefixes.zip -O scripts/nonbreaking_prefixes.zip
!unzip scripts/nonbreaking_prefixes.zip -d scripts/

In [ ]:
!wget https://ilk.uvt.nl/~shterion/data/EN-NL.zip -O EN-NL.zip
!unzip -K EN-NL.zip

Let's see what we have:

In [ ]:
!head -10 EN-NL/data/data.*

I will continue using the EN-NL, but you can choose whichever you prefer. 

3. Tokenize the data (takes some time)

In [ ]:
!ls -l
!perl scripts/tokenizer.perl -no-escape -l en -threads 16 < EN-NL/data/data.en > EN-NL/data/data.tok.en
!perl scripts/tokenizer.perl -no-escape -l nl -threads 16 < EN-NL/data/data.nl > EN-NL/data/data.tok.nl

4. Now we need to split the data into train test and development sets. We can do this with the `train_test_dev.py` script provided at: https://ilk.uvt.nl/~shterion/scripts/train_test_dev.py which we downloaded earlier and stored in the `scripts` folder.

In [ ]:
%run scripts/train_test_dev.py --source=EN-NL/data/data.tok.nl --target=EN-NL/data/data.tok.en 

Ideally, now that we have all the data split into train test and dev, tokenized and cleaned we would need to perform one more preprocessing - to split it into subword units. However, due to the lack of time we will not do that.

You can do this at home using the provided `bpe.sh` script (https://ilk.uvt.nl/~shterion/scripts/bpe.sh)

5. Now we need to build a vocabulary - a list of words and their numerical representations that will be used by the NNs. We can do this by running the `onmt_build_vocab` using the configuration file provided with the data in the config folder: `config_vocab.yaml`. 

! If you are using BPE make sure that the files you use to build the vocabulary have the correct extension. For simplicity, I have provided the `config_vocab_bpe.yaml` file.

In [ ]:
!onmt_build_vocab -config EN-NL/config/config_vocab.yaml

Let's see what do we have in the vocabulary:

In [ ]:
!wc -l /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg
!head /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg

In [ ]:
import csv
import sys
word_lengths = []
csv.field_size_limit(sys.maxsize)

i = 0

with open("/content/drive/MyDrive/SE_personal/EN-NL/data/vocab.trg") as tsv:
    for line in csv.reader(tsv, delimiter="\t"):
      n_words += 1
      word_lengths.append(len(line[0]))  



average = sum(word_lengths) / len(word_lengths)
print(average)
  

6. Now we can start the training process - takes a loooong time! Be patient! 

In [ ]:
!onmt_train -config /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/config/config_train_rnn.yaml -train_from EN-NL/models/model_step_460000.pt

7. Now that the training is over we can translate using the model. 
You can use the `config_trans.yaml` file provided in the config folder. 
It will translate the test set with the first saved model. Change these to any arbitrary file you wish to translate and to any model you would like to use.

In [ ]:
!onmt_translate -config /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/config/config_trans.yaml